# Export trades ticks and immediately preceding book ticks using dedicated service - Python

### Overview
Here we want to extract trade ticks and also know about the state of the book immediately before each trade.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

### Inputs/outputs
Extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    
***

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio systemathics.apis

In [ ]:
import grpc
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2 as tick_trades_and_book
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2_grpc as tick_trades_and_book_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'INTC' # 'AMD'
exchange = 'XNGS' # 'BATS'

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2022, month = 1, day = 14), 
    end_date = date.Date(year = 2022, month = 1, day = 14)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 14, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 23, minutes = 0, seconds = 0)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

The following code snippets create the market data request and instantiate the service:

In [ ]:
# generate the tick trades and book request
identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades_and_book.TickTradesAndBookRequest(
    identifiers = [identifier],
    constraints = constraints
)

#### 3.4 Export data
Tick by tick data is sent through a *stream*, the following code snippet iterates ticks and keep trades ticks and their immediately preceding book tick (if any)

Output the results to CSV


In [ ]:
import csv
import systemathics.apis.type.shared.v1.trade_and_book_pb2 as trade_and_book_pb2
filename = 'trades_and_immediately_preceding_bbo_{0}-{1}.csv'.format(ticker, exchange)


with open(filename, mode='w') as normalized_ticks_file:
    normalized_ticks_writer = csv.writer(normalized_ticks_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    normalized_ticks_writer.writerow(["time_stamp", "datetime.seconds", "datetime.nanos", "bid_size", "bid_price", "ask_price", "ask_size", "trade_price", "trade_size", "not_shown_book_ticks_before"])

    csv_lines_count = 0
    csv_book_lines_count = 0
    csv_trade_lines_count = 0

    previous = trade_and_book_pb2.TradeAndBook()
    not_shown_book_ticks_before = 0
    total_ticks = 0
    total_trade_ticks = 0
    total_book_ticks = 0


    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:  
                
            # instantiate the tick trades and book service
            service = tick_trades_and_book_service.TickTradesAndBookServiceStub(channel)
            for current in service.TickTradesAndBook(request = request, metadata = [('authorization', token)]):
                if (current.data.trade.time_stamp.seconds > 0):
                    total_ticks += 1 
                    total_trade_ticks += 1
                
                if (current.data.book.time_stamp.seconds > 0): 
                    total_ticks += 1
                    total_book_ticks += 1

                if (current.data.trade.time_stamp.seconds > 0):     
                    if (previous.data.book.time_stamp.seconds > 0):
                        seconds = previous.data.book.time_stamp.seconds
                        nanos = previous.data.book.time_stamp.nanos
                        book_time = datetime.fromtimestamp(seconds)
                        book_time_ms = round(nanos/1000000)
                        bid_size = previous.data.book.bid.size
                        bid_price = previous.data.book.bid.price
                        ask_price = previous.data.book.ask.price
                        ask_size = previous.data.book.ask.size
                        csv_lines_count += 1
                        csv_book_lines_count += 1
                        normalized_ticks_writer.writerow([f"{book_time}.{book_time_ms}", seconds, nanos,  bid_size, bid_price, ask_price, ask_size, "", "", not_shown_book_ticks_before - 1 if not_shown_book_ticks_before > 0 else ""])
  
                    seconds = current.data.trade.time_stamp.seconds
                    nanos = current.data.trade.time_stamp.nanos
                    trade_time = datetime.fromtimestamp(seconds)
                    trade_time_ms = round(nanos/1000000)
                    trade_price = current.data.trade.price
                    trade_size = current.data.trade.size
                    csv_lines_count += 1
                    csv_trade_lines_count += 1
                    normalized_ticks_writer.writerow([f"{trade_time}.{trade_time_ms}", seconds, nanos, "", "", "", "", trade_price, trade_size, ""])
                    last_is_trade = True
                    not_shown_book_ticks_before = 0
                else:
                    last_is_trade = False
                    not_shown_book_ticks_before += 1
                    
                previous = current    

    except grpc.RpcError as e:
        print(e.code().name)
        print(e.details())

print(f"#total_ticks={total_ticks:,} #total_book_ticks={total_book_ticks:,} #total_trade_ticks={total_trade_ticks:,}")
print(f"#csv_lines_count={csv_lines_count:,} #csv_book_lines_count={csv_book_lines_count:,} #csv_trade_lines_count={csv_trade_lines_count:,} [{filename}]")